In [105]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

def live_plot(data_dict, figsize=(7,5), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label,data in data_dict.items():
        plt.plot(data, label=label)
    plt.title(title)
    plt.grid(True)
    plt.xlabel('epoch')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();

error = {   
        'TE01':'Type Error: Requires format = iterble of size 2',
        'TE02':'Type Error: Requires format = float',
        'VE01':'Value Error: Distance has a value of 0'
        }


class ball():
    def __init__(self, position = tuple, velocity = tuple, mass = float) -> None:
        if len(position) != 2:
            raise TypeError(error['TE01'])
        else:
            self._position = np.array(position)
        if len(position) != 2:
            raise TypeError(error['TE01'])
        else:
            self._velocity = np.array(velocity)
        if type(mass) == float or type(mass) == int:
            self._mass = float(mass)
        else:
            raise TypeError(error['TE02'])

    @property
    def position(self):
        return self._position
    
    @position.setter
    def position(self, position = tuple):
        if len(position) != 2:
            raise TypeError(error['TE01'])
        else:
            self._position = np.array(position)

    @property
    def velocity(self):
        return self._velocity

    @velocity.setter
    def velocity(self, velocity = tuple):
        if len(velocity) != 2:
            raise TypeError(error['TE01'])
        else:
            self._velocity = np.array(velocity)               
    
    @property
    def mass(self):
        return self._mass

    @mass.setter
    def mass(self, mass):
        if type(mass) != float:
            raise TypeError(error['TE02'])
        else:
            self._mass = float(mass)       

    def force_between(self, ball, scale = 6.6743 * 10**-11):
        dist = np.linalg.norm(self._position-ball.position)
        vector = ball.position - self._position
        if vector[0] == 0:
            return 0
        alpha_prime = vector[1]/vector[0]
        force = scale*self._mass*ball.mass/np.square(dist)
        return force*np.array((np.cos(np.arctan(alpha_prime)),np.sin(np.arctan(alpha_prime))))
    
    def acceleration_to(self, ball, scale):
        return self.force_between(ball, scale)/self._mass
    
    def update(self,ball,scale, time_step):
        self._velocity = self._velocity + self.acceleration_to(ball,scale) * time_step
        self._position = self._position + self._velocity * time_step
        ball.velocity = ball.velocity - ball.acceleration_to(self, scale) * time_step
        ball.position = ball.position + ball.velocity * time_step
        return self._position, self._velocity, ball.position, ball.velocity
    
def update(balls):
    while True:
        pass

In [106]:
ball1 = ball((0,0),(0,0),1)
ball2 = ball((2,5),(0,0),1)

balls = [ball1, ball2]


time_step = 0.1

# while balls:
#     primary_ball = balls.pop()
#     print(primary_ball._position)

In [107]:
while True:
    clear_output(wait=True)
    position, velocity, position_ball, velocity_ball = ball1.update(ball2, scale, time_step)
    print(velocity, velocity_ball)
    plt.figure(figsize=(12,6))
    plt.scatter([ball1.position[0], ball2.position[0]], [ball1.position[0], ball2.position[0]])
    plt.grid(True)
    plt.xlabel('epoch')
    plt.show();

[2.65208925 6.63022314] [-2.44201898 -6.10504746]


KeyboardInterrupt: 